In [ ]:
from fastapi import FastAPI
import snowflake.connector
import pandas as pd
import uvicorn

# Create FastAPI instance
app = FastAPI()

# Snowflake connection configuration
config = {
    "account": "SFEDU02-KFB85562",
    "user": "FOX",
    "password": "Arjun9481@",
    "role": "TRAINING_ROLE",
    "warehouse": "ANIMAL_TASK_WH",
    "database": "CATCH_ME_IF_YOU_CAN",
    "schema": "PUBLIC"
}

# Function to execute Snowflake queries
def execute_snowflake_query(query):
    conn = snowflake.connector.connect(**config)
    cur = conn.cursor()
    cur.execute(query)
    df = cur.fetch_pandas_all()
    cur.close()
    conn.close()
    return df.to_dict(orient="records")


def create_materialized_views():
    conn = snowflake.connector.connect(**config)
    cur = conn.cursor()

    queries = {
        "table_usage": """
        CREATE MATERIALIZED VIEW IF NOT EXISTS mv_table_usage AS
        WITH exploded_tables AS (
            SELECT
                query_id,
                num_joins,
                t.value AS table_id
            FROM
                live_queries,
                LATERAL FLATTEN(INPUT => SPLIT(read_table_ids, ',')) t
            WHERE
                read_table_ids IS NOT NULL
        )
        SELECT
            table_id,
            COUNT(*) AS total_count,
            COUNT(CASE WHEN num_joins > 0 THEN 1 ELSE NULL END) AS join_count,
            COALESCE(COUNT(CASE WHEN num_joins > 0 THEN 1 ELSE NULL END) * 1.0 / COUNT(*), 0) AS join_usage_ratio
        FROM
            exploded_tables
        GROUP BY
            table_id;
        """,

        "user_weekly_spending": """
        CREATE MATERIALIZED VIEW IF NOT EXISTS mv_user_weekly_spending AS
        SELECT 
            user_id,
            DATE_PART('week', arrival_timestamp) AS week,
            DATE_PART('year', arrival_timestamp) AS year,
            AVG(
                (COALESCE(mbytes_scanned, 0) * 0.01) +  
                (COALESCE(execution_duration_ms, 0) * 0.00001) +  
                (CASE WHEN write_table_ids IS NOT NULL THEN 0.02 ELSE 0 END)  
            ) AS avg_weekly_spending
        FROM 
            live_queries
        GROUP BY 
            user_id, DATE_PART('week', arrival_timestamp), DATE_PART('year', arrival_timestamp);
        """,

        "query_performance": """
        CREATE MATERIALIZED VIEW IF NOT EXISTS mv_query_performance AS
        SELECT
            query_type,
            COUNT(*) AS query_count,
            SUM(execution_duration_ms) AS total_execution_time_ms,
            AVG(execution_duration_ms) AS average_execution_time_ms
        FROM
            live_queries
        GROUP BY
            query_type;
        """
    }

    for name, query in queries.items():
        cur.execute(query)
        print(f" Materialized View `{name}` created successfully.")

    cur.close()
    conn.close()

# Run the materialized view creation function once
create_materialized_views()


@app.get("/api/table-usage")
async def get_table_usage():
    query = """
    SELECT * FROM mv_table_usage 
    ORDER BY total_count DESC;
    """
    result = execute_snowflake_query(query)
    return {"table_usage": result}

@app.get("/api/user-weekly-spending")
async def get_user_weekly_spending():
    query = """
    SELECT * FROM mv_user_weekly_spending 
    ORDER BY user_id, year, week;
    """
    result = execute_snowflake_query(query)
    return {"user_weekly_spending": result}

@app.get("/api/query-performance")
async def get_query_performance():
    query = """
    SELECT * FROM mv_query_performance 
    ORDER BY total_execution_time_ms DESC;
    """
    result = execute_snowflake_query(query)
    return {"query_performance": result}

# Run the FastAPI service
if __name__ == "__main__":
    uvicorn.run("main:app", host="127.0.0.1", port=8000, reload=True)


INFO:     Will watch for changes in these directories: ['C:\\Users\\prasa']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [12092] using StatReload
